In [2]:
using GLMakie
using Measures
using Oceananigans
using Printf
using JLD2

In [3]:
setname = "U250N100Lz100g100"

include("../parameters.jl")

pm = getproperty(SimParams(), Symbol(setname))

pm = merge(pm, (; Tanθ = sqrt((pm.σ^2 - pm.f^2)/(pm.Ñ^2-pm.σ^2)),
                Tanα = pm.γ * sqrt((pm.σ^2 - pm.f^2)/(pm.Ñ^2-pm.σ^2)),
                nz = round(Int,pm.Lz/2),
                m = -π/pm.Lz,
                l = sqrt(((π/pm.Lz)^2 * (pm.f^2 - pm.σ^2)) / (pm.σ^2 - pm.Ñ^2)),
                Tf = 2*π/pm.f, 
                Tσ = 2*π/pm.σ))

const gausT_center = 895                                 # gaussian paramereters for curved "corner" of slope
const gausT_width = 180
const ySlopeSame = 1336.6                           # point where planar and curved corner math up the best

ΔySlopeSame = 0


@inline heaviside(X) = ifelse(X < 0, 0., 1.) # heaviside returns 1 if x >= 0
# exponential gaussian for curved corner
@inline expcurve(y, ystar, smo) = -pm.Lz + pm.Lz * exp( - (y - ystar)^2 / (2*smo^2))
# planar slope line
@inline linslope(y) = -pm.Tanα * y
# combining the 2 with heaviside split at ySlopeSame
@inline curvedslope(y) = @. linslope(y) + (-linslope(y) + expcurve(y, gausT_center-ΔySlopeSame, gausT_width)) * heaviside(y-ySlopeSame)


curvedslope (generic function with 1 method)

In [4]:
path_name = "../../../../"
filepath = path_name * "DyeU250sonly.jld2"
f2 = jldopen(filepath)

JLDFile /Users/vicwhit/Desktop/DyeU250sonly.jld2 (read-only)
 └─🔢 c2_timeseries

In [5]:
xc = 2:4:150
yc = 2:4:5498
zc = -499:2:-1
land = [curvedslope(y) for x in -5:4:155, y in 0:4:3500];

In [6]:
c_timeseries = f2["c2_timeseries"];

┌ Warning: read type ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded, RectilinearGrid{Float64, Periodic, Bounded, Bounded, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}, CPU}, GridFittedBoundary{Field{Center, Center, Center, Nothing, RectilinearGrid{Float64, Periodic, Bounded, Bounded, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePreci

In [7]:
f = Figure(resolution = (1500, 550),fontsize=26) 
ga = f[1, 1] = GridLayout()

ax1 = Axis3(ga[1, 1] , azimuth = π/8, # rotation of plot
            elevation = 0.15, 
            aspect =(1,3,1), xtickwidth = 0,
            xticks = ([2,150], ["152", "0"]), xticklabelpad = -25, 
            zticks = ([-500, -250, 0]),
            yticks = ([1000, 2000, 3000]),
            xlabeloffset = 10, ylabeloffset = 30, zlabeloffset = 100,
            xlabel="x [m]", ylabel = "y [m]", zlabel = "z [m]",)
limits!((0,152), (0,3500), (-500,0))
surface!(ax1, -5:4:155, yc, land, shading = true, color=fill((:black, .7),100,100),)

# topography yz cut lower and uppbounds
lower = Point3f.(152, yc, -500);
upper = Point3f.(152, yc, curvedslope.(yc));
band!(ax1, lower, upper, color = (:black, 0.9), shading = true)

display(f)

GLMakie.Screen(...)

In [8]:
n = Observable(1)
title = @lift "Internal Wave Breaking, t = " * string(round(c_timeseries.times[$n], digits=2)) * ", Tσ = "*string(round(c_timeseries.times[$n]/pm.Tσ, digits=2))
Label(ga[1, 1, Top()], title, valign = :bottom, font = :bold, padding = (0, 0, 5, 0))

c = @lift log10.(clamp.(c_timeseries.data.parent[:,:,:,$n], 7e-5,1));

cg = contour!(ax1, xc, yc, zc, c, levels = (-4:0.05:0), colormap = :thermal)

Combined{Makie.contour, Tuple{StepRange{Int64, Int64}, StepRange{Int64, Int64}, StepRange{Int64, Int64}, Array{Float32, 3}}}

In [9]:
cb = Colorbar(ga[1, 2], limits = (-4,0), ticks = (-4:1:0, ["10⁻⁴", "10⁻³", "10⁻²", "10⁻¹", "10⁰"] ),
    label = "Tracer Concentration", lowclip = :white, colormap = :thermal, size = 25)
cb.alignmode = Mixed(right = 0)

Mixed(GridLayoutBase.RectSides{Union{Nothing, Float32, GridLayoutBase.Protrusion}}(nothing, 0.0f0, nothing, nothing))

In [10]:
savename = "slope3dwaveseries_" * setname
apath  =  "Plots/"
frames = 1:length(c_timeseries.times)
record(f, apath * savename * ".mp4", frames, framerate=8) do j
    msg = string("Plotting frame ", j, " of ", frames[end])
    print(msg * " \r")
    n[] = j
end

"Plots/slope3dwaveseries_U250N100Lz100g100.mp4"